Data Date Range:
    <br />Start time: 2016-05-05 09:40:26
    <br />End time: 2018-01-17 01:59:42

#### This notebook generates the following table:
- actions_alg: a table of key interactions, and corresponding information about the user and the movie
- interactions_rec_ratio: the ratio of key interactions over rec clicks, broken down by algorithms

#### Explanations of selected actions:
A list of the actions:

    ['TrailerModalLaunched', 'PlayedFromLaunch', 'PlayedFromNext',
    'PlayedFromRecommendation', 'MovieDetailsModalOpened',
    'TrailerPaused', 'MovieDetailsModalClosed', 'TrailerModalShutdown',
    'IssueRaised', 'PlayedFromYoutubePlayer', 'TrailerEnded',
    'TrailerLiked', 'GoToMovieDetails', 'RecommendationsRefreshed',
    'TrailerUnliked', 'TrailerDisliked', 'TrailerUndisliked',
    'WishlistedMovie', 'UnwishlistedMovie', 'PlayedFromPrevious',
    'RatedMovie']
       
##### If a user opens the trailer from the movie detail page: ####
* TrailerModalLaunched: If a user opens the trailer from the movie detail page
* PlayedFromLaunch: Happens right after 'TrailerModalLaunched'

##### User clicked in the trailer interface to go to a different trailer recommended: ####
* PlayedFromNext
* PalyedFromPrevious
* PlayedFromRecommendation


### Import and run code from another notebook

In [2]:
import pandas as pd
import numpy as np

In [5]:
%run '1 - Trailers Viewed and Aggregate Table, by Algorithm.ipynb'


FileNotFoundError: File b'../Clean_Data/log_trailer_actions_sessionIds.csv' does not exist

### Generate the dataset with the counts of each action  ###

Except for the actions "PlayedFromLaunch","PlayedFromNext", "PlayedFromRecommendation", "PlayedFromPrevious" that indicate if a trailer is played or not, there are also other informative actions. Therefore, in this section, we count how many times each action happened. 

In [25]:
action_data = pd.read_csv('../Clean_Data/log_trailer_actions_sessionIds.csv')

# Get the algorithm assignment
expt_trailer = pd.read_csv('../Clean_Data/expt_trailer.csv', sep=',')
alg_assignment = expt_trailer[['userId','loginId','recommenderId']]

In [18]:
# generate the dataset with the counts of each action
actions_alg = pd.merge(action_data, alg_assignment, on=['loginId','userId'], how='left')
actions_alg.drop(['Unnamed: 0'],axis=1, inplace=True)
interactions_alg = actions_alg[actions_alg['action'].isin(['TrailerLiked','TrailerDisliked','MovieDetailsModalOpened','WishlistedMovie','GoToMovieDetails'])]

In [19]:
interactions_alg.groupby('action').count()

,userId,loginId,movieId,tstamp,position,sessionIds,recommenderId
action,,,,,,,
GoToMovieDetails,987,987,987,987,987,987,887
MovieDetailsModalOpened,6718,6718,6718,6718,6718,6718,6006
TrailerDisliked,998,998,998,998,998,998,901
TrailerLiked,2260,2260,2260,2260,2260,2260,2096
WishlistedMovie,3164,3164,3164,3164,3164,3164,2825


#### Find 'WatchedMoreThanHalf'

In [26]:
# Data ends on 2018-01-17
watch = pd.read_csv('../Clean_Data/log_trailer_watch.csv')
watch_small = watch[watch['tstamp']<"2018-01-17 01:59:42"]

mlusers = [32451,49002,49253,50540,51234,51714,65817,66644,68903,70023,70026,92721,92723,95195,96279,96483,96691,97397,104218,113699,114092,114844,122798,124450,124790,125283,125431,127628,127979,129299,130353,132131,132145,132637,132726,132728,132871,133047,134070,134088,134143,134235,134342,134349,134372,134373,134393,134484,134496,134498,134504,134533,135260,135670,136346,136505,136995,137096,137331,137372,138083,138089,138090,138108,141799,142115,143473,149815,151709,151982,166008,173637,176970,183038,183567,197881,202657,207221,221192,221515,225510,233659,233985,237287,237332,238576,238876,240312,242238,242683,243010,243445,245696,246018,246058,246587,246828,248583,251610,251922,252238,252534,253066,253866,253869,257628,257629,257756,258553,258554,258555,258556,258557,258558,258559,258560,258561,258562,258563,258564,258565,259056,260701,264178,266542,268253,269945,286014,287206,292397,301040,304542,313322,316070,317416,318334]

watch_noml = watch[~watch.userId.isin(mlusers)].copy()

In [27]:
watch_noml.sort_values('percentWatched',ascending=False, inplace=True)

In [28]:
watch_noml = watch_noml[watch_noml['percentWatched']<=100].copy()

In [29]:
watch_noml.sort_values(by=['loginId','movieId'],inplace=True)

In [30]:
# Keep only the largest percent watched.
watch_noml_small = watch_noml.drop_duplicates(subset=["loginId","movieId"], keep="first").copy()
# watch_noml_small.groupby('percentWatched').count()
watch_noml_small.head()

,userId,loginId,movieId,youtubeTrailerId,percentWatched,positionInList,tstamp
116436,276159,008C57f,118985,2xD9uTlh5hI,90,0,2016-09-12 13:55:20
588111,305068,009NNgE,34162,ZeUSo8voIXM,45,0,2017-11-25 12:22:32
556148,314354,009PCM1,1222,x9f6JaaX7Wg,90,0,2017-10-29 17:22:16
589454,304970,00AmPNC,133419,OgPm-yaLoyo,60,0,2017-11-26 03:46:48
70845,233888,00Et4vt,1466,rsBx_eTVSpQ,30,0,2016-07-22 05:09:54


In [31]:
watch_more_than_half = watch_noml_small[watch_noml_small['percentWatched'] >= 50]
watch_more_than_half.count()

userId              101059
loginId             101059
movieId             101059
youtubeTrailerId    101059
percentWatched      101059
positionInList      101059
tstamp              101059
dtype: int64

In [32]:
watch = pd.read_csv('../Clean_Data/log_trailer_watch.csv')

# Exclude movielens users
mlusers = [32451,49002,49253,50540,51234,51714,65817,66644,68903,70023,70026,92721,92723,95195,96279,96483,96691,97397,104218,113699,114092,114844,122798,124450,124790,125283,125431,127628,127979,129299,130353,132131,132145,132637,132726,132728,132871,133047,134070,134088,134143,134235,134342,134349,134372,134373,134393,134484,134496,134498,134504,134533,135260,135670,136346,136505,136995,137096,137331,137372,138083,138089,138090,138108,141799,142115,143473,149815,151709,151982,166008,173637,176970,183038,183567,197881,202657,207221,221192,221515,225510,233659,233985,237287,237332,238576,238876,240312,242238,242683,243010,243445,245696,246018,246058,246587,246828,248583,251610,251922,252238,252534,253066,253866,253869,257628,257629,257756,258553,258554,258555,258556,258557,258558,258559,258560,258561,258562,258563,258564,258565,259056,260701,264178,266542,268253,269945,286014,287206,292397,301040,304542,313322,316070,317416,318334]

watch_small = watch[watch['tstamp']<"2018-01-17 01:59:42"]
watch_noml = watch_small[~watch_small.userId.isin(mlusers)]
watch_noml = watch_noml.drop(['youtubeTrailerId'],axis=1).copy()

In [33]:
watch_noml_small = watch_noml[watch_noml['percentWatched']<=100].copy()
watch_noml_small.sort_values('percentWatched',ascending=False, inplace=True)
watch_noml_small.sort_values(by=['loginId','movieId'],inplace=True)

In [34]:
watch_nodup = watch_noml_small.drop_duplicates(subset=["loginId","movieId"], keep="first").copy()

In [35]:
# Keep only the highest percent watched for each trailer view
watch_nodup.groupby('percentWatched').count()

,userId,loginId,movieId,positionInList,tstamp
percentWatched,,,,,
15,13802,13802,13802,13802,13802
16,715,715,715,715,715
17,49,49,49,49,49
18,4,4,4,4,4
19,12,12,12,12,12
20,1,1,1,1,1
21,1,1,1,1,1
22,1,1,1,1,1
24,1,1,1,1,1


In [37]:
expt_trailer = pd.read_csv('../Clean_Data/expt_trailer.csv')
watch_alg = pd.merge(watch_nodup, expt_trailer, on=['loginId','userId'], how="inner") # Use inner to drop sessions without alg assignment

In [38]:
watch_more_than_half = watch_alg[watch_alg['percentWatched'] >= 50].copy()

In [39]:
watch_more_than_half.count()

userId            92226
loginId           92226
movieId           92226
percentWatched    92226
positionInList    92226
tstamp_x          92226
recommenderId     92226
tstamp_y          92226
dtype: int64

In [40]:
watch_more_than_half_small = watch_more_than_half[['loginId','movieId','recommenderId','userId','positionInList','tstamp_x']].copy()
watch_more_than_half_small['action']='WatchedMoreThanHalf'
watch_more_than_half_small.rename(columns={"positionInList":"position",'tstamp_x':'tstamp'}, inplace=True)

In [41]:
interactions_alg = interactions_alg.append(watch_more_than_half_small)

/Users/evazhong/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [42]:
interactions_alg.groupby("action").count()

,loginId,movieId,position,recommenderId,sessionIds,tstamp,userId
action,,,,,,,
GoToMovieDetails,987,987,987,887,987,987,987
MovieDetailsModalOpened,6718,6718,6718,6006,6718,6718,6718
TrailerDisliked,998,998,998,901,998,998,998
TrailerLiked,2260,2260,2260,2096,2260,2260,2260
WatchedMoreThanHalf,92226,92226,92226,92226,0,92226,92226
WishlistedMovie,3164,3164,3164,2825,3164,3164,3164


#### Finish Appending watch_more_than_half to actions_alg  ⤴️

### Interactions with recommended trailers clicked

In [43]:
interactions_alg.count()

action           106353
loginId          106353
movieId          106353
position         106353
recommenderId    104941
sessionIds        14127
tstamp           106353
userId           106353
dtype: int64

In [1]:
interactions_alg.to_csv('../Clean_Data/actions_alg.csv', sep=',')

NameError: name 'interactions_alg' is not defined

In [44]:
actions_alg_rec = actions_alg[actions_alg['action'].isin(["PlayedFromNext", "PlayedFromRecommendation", "PlayedFromPrevious"])]
actions_alg_rec_small = actions_alg_rec[['loginId','movieId','action','tstamp','userId','recommenderId']]
actions_alg_rec_small.head()

,loginId,movieId,action,tstamp,userId,recommenderId
34,00bSwKf,159193,PlayedFromRecommendation,2017-04-29 07:07:09,304211,FilmReleaseDate
66,00bSwKf,132800,PlayedFromRecommendation,2017-04-29 07:15:57,304211,FilmReleaseDate
70,00bSwKf,161966,PlayedFromRecommendation,2017-04-29 07:18:04,304211,FilmReleaseDate
83,00bSwKf,5617,PlayedFromRecommendation,2017-04-29 07:18:16,304211,FilmReleaseDate
87,00bSwKf,113829,PlayedFromRecommendation,2017-04-29 07:18:34,304211,FilmReleaseDate


In [45]:
# Actions taken place AFTER user clicked on the recommended trailer
interactions_alg_agg = pd.merge(interactions_alg, actions_alg_rec_small, on=['loginId','movieId'],how='inner')
interactions_alg_agg.count()

action_x           7055
loginId            7055
movieId            7055
position           7055
recommenderId_x    6812
sessionIds         2862
tstamp_x           7055
userId_x           7055
action_y           7055
tstamp_y           7055
userId_y           7055
recommenderId_y    6812
dtype: int64

In [46]:
# Count the number of times an action taken place AFTER user clicked on a recommended trailer
interactions_alg_agg_small = interactions_alg_agg[interactions_alg_agg['tstamp_x']>interactions_alg_agg['tstamp_y']].copy()


interactions_alg_group = interactions_alg_agg_small.groupby(['action_x','recommenderId_x']).count().reset_index()[['action_x','recommenderId_x','loginId']]
interactions_alg_group.rename(columns = {"action_x":"action", "recommenderId_x":"recommenderId","loginId":"count"},inplace=True)

In [47]:
# Action_alg_rec_small is created abooove

actions_alg_rec_agg = actions_alg_rec_small.groupby('recommenderId').count().reset_index()
actions_alg_rec_agg = actions_alg_rec_agg[['recommenderId','loginId']].copy()
actions_alg_rec_agg.rename(columns={"loginId":"count_rec_clicks"},inplace=True)

In [48]:
actions_alg_rec_agg.head()

,recommenderId,count_rec_clicks
0,FilmReleaseDate,2801
1,PredictedRating,2189
2,ShuffledTopPicks,1367
3,TagSimilarity,2780


In [49]:
interactions_alg_rec_percent = pd.merge(interactions_alg_group, actions_alg_rec_agg, on="recommenderId", how="left")
interactions_alg_rec_percent['ratio_action_rec_clicks'] = interactions_alg_rec_percent['count']/interactions_alg_rec_percent['count_rec_clicks']

In [50]:
interactions_alg_rec_percent

,action,recommenderId,count,count_rec_clicks,ratio_action_rec_clicks
0,GoToMovieDetails,FilmReleaseDate,68,2801,0.024277
1,GoToMovieDetails,PredictedRating,66,2189,0.030151
2,GoToMovieDetails,ShuffledTopPicks,18,1367,0.013168
3,GoToMovieDetails,TagSimilarity,67,2780,0.024101
4,MovieDetailsModalOpened,FilmReleaseDate,289,2801,0.103177
5,MovieDetailsModalOpened,PredictedRating,244,2189,0.111466
6,MovieDetailsModalOpened,ShuffledTopPicks,67,1367,0.049012
7,MovieDetailsModalOpened,TagSimilarity,263,2780,0.094604
8,TrailerDisliked,FilmReleaseDate,14,2801,0.004998
9,TrailerDisliked,PredictedRating,8,2189,0.003655


In [51]:
interactions_alg_rec_percent.to_csv('../Clean_Data/interactions_rec_ratio_9_15.csv')


### Code above are run in September 2018. Because the data in the database changed, results from code above are slightly different.
### Codes below are written and run in May, 2018. Don't edit them. Leave them for comparison.

In [49]:
# Actions taken place AFTER user clicked on the recommended trailer
interactions_alg_agg = pd.merge(interactions_alg, actions_alg_rec_small, on=['loginId','movieId'],how='inner')
interactions_alg_agg.count()

action_x           7065
loginId            7065
movieId            7065
position           7065
recommenderId_x    6822
sessionIds         2871
tstamp_x           7065
userId_x           7065
action_y           7065
tstamp_y           7065
userId_y           7065
recommenderId_y    6822
dtype: int64

In [104]:
# Count the number of times an action taken place AFTER user clicked on a recommended trailer
interactions_alg_agg_small = interactions_alg_agg[interactions_alg_agg['tstamp_x']>interactions_alg_agg['tstamp_y']].copy()


interactions_alg_group = interactions_alg_agg_small.groupby(['action_x','recommenderId_x']).count().reset_index()[['action_x','recommenderId_x','loginId']]
interactions_alg_group.rename(columns = {"action_x":"action", "recommenderId_x":"recommenderId","loginId":"count"},inplace=True)

In [106]:
# Action_alg_rec_small is created abooove

actions_alg_rec_agg = actions_alg_rec_small.groupby('recommenderId').count().reset_index()
actions_alg_rec_agg = actions_alg_rec_agg[['recommenderId','loginId']].copy()
actions_alg_rec_agg.rename(columns={"loginId":"count_rec_clicks"},inplace=True)

In [107]:
actions_alg_rec_agg.head()

,recommenderId,count_rec_clicks
0,FilmReleaseDate,2802
1,PredictedRating,2191
2,ShuffledTopPicks,1369
3,TagSimilarity,2780


In [108]:
interactions_alg_rec_percent = pd.merge(interactions_alg_group, actions_alg_rec_agg, on="recommenderId", how="left")
interactions_alg_rec_percent['ratio_action_rec_clicks'] = interactions_alg_rec_percent['count']/interactions_alg_rec_percent['count_rec_clicks']

In [109]:
interactions_alg_rec_percent

,action,recommenderId,count,count_rec_clicks,ratio_action_rec_clicks
0,GoToMovieDetails,FilmReleaseDate,68,2802,0.024268
1,GoToMovieDetails,PredictedRating,66,2191,0.030123
2,GoToMovieDetails,ShuffledTopPicks,18,1369,0.013148
3,GoToMovieDetails,TagSimilarity,67,2780,0.024101
4,MovieDetailsModalOpened,FilmReleaseDate,289,2802,0.103141
5,MovieDetailsModalOpened,PredictedRating,244,2191,0.111365
6,MovieDetailsModalOpened,ShuffledTopPicks,68,1369,0.049671
7,MovieDetailsModalOpened,TagSimilarity,263,2780,0.094604
8,TrailerDisliked,FilmReleaseDate,14,2802,0.004996
9,TrailerDisliked,PredictedRating,8,2191,0.003651


In [34]:
interactions_alg_rec_percent.to_csv('../Clean_Data/interactions_rec_ratio.csv')


### Interactions with all trailers viewed

In [106]:
# We need to double check this section if we need this data

In [98]:
interactions_alg.count()

action           106382
loginId          106382
movieId          106382
position         106382
recommenderId    104961
sessionIds        14156
tstamp           106382
userId           106382
dtype: int64

In [99]:
interactions_alg.groupby("action").count()

,loginId,movieId,position,recommenderId,sessionIds,tstamp,userId
action,,,,,,,
GoToMovieDetails,989,989,989,889,989,989,989
MovieDetailsModalOpened,6745,6745,6745,6024,6745,6745,6745
TrailerDisliked,998,998,998,901,998,998,998
TrailerLiked,2260,2260,2260,2096,2260,2260,2260
WatchedMoreThanHalf,92226,92226,92226,92226,0,92226,92226
WishlistedMovie,3164,3164,3164,2825,3164,3164,3164


In [100]:
actions_alg_total = actions_alg[actions_alg['action'].isin(["PlayedFromLaunch", "PlayedFromNext", "PlayedFromRecommendation", "PlayedFromPrevious"])]
actions_alg_total_small = actions_alg_total[['loginId','movieId','action','tstamp','userId','recommenderId']]

In [101]:
# Actions taken place AFTER user clicked on the recommended trailer
interactions_alg_total_agg = pd.merge(interactions_alg, actions_alg_total_small, on=['loginId','movieId'],how='inner')
interactions_alg_total_agg.drop_duplicates(["loginId","movieId","tstamp_x"],inplace=True)

In [102]:
interactions_alg_total_agg_small = interactions_alg_total_agg[interactions_alg_total_agg['tstamp_x']>interactions_alg_total_agg['tstamp_y']].copy()
interactions_alg_total_agg_small.count()

action_x           100377
loginId            100377
movieId            100377
position           100377
recommenderId_x     99542
sessionIds           8622
tstamp_x           100377
userId_x           100377
action_y           100377
tstamp_y           100377
userId_y           100377
recommenderId_y     99542
dtype: int64

In [103]:
interactions_alg_total_agg_small.groupby("action_x").count()
# We excluded sessions with no alg assignment!

,loginId,movieId,position,recommenderId_x,sessionIds,tstamp_x,userId_x,action_y,tstamp_y,userId_y,recommenderId_y
action_x,,,,,,,,,,,
GoToMovieDetails,419,419,419,381,419,419,419,419,419,419,381
MovieDetailsModalOpened,2579,2579,2579,2324,2579,2579,2579,2579,2579,2579,2324
TrailerDisliked,905,905,905,818,905,905,905,905,905,905,818
TrailerLiked,2083,2083,2083,1941,2083,2083,2083,2083,2083,2083,1941
WatchedMoreThanHalf,91755,91755,91755,91755,0,91755,91755,91755,91755,91755,91755
WishlistedMovie,2636,2636,2636,2323,2636,2636,2636,2636,2636,2636,2323


In [111]:
# Count the number of times an action taken place AFTER user clicked on a recommended trailer
interactions_alg_total_agg_small = interactions_alg_total_agg[interactions_alg_total_agg['tstamp_x']>interactions_alg_total_agg['tstamp_y']].copy()


interactions_alg_total_group = interactions_alg_total_agg_small.groupby(['action_x','recommenderId_x']).count().reset_index()[['action_x','recommenderId_x','loginId']]
interactions_alg_total_group.rename(columns = {"action_x":"action", "recommenderId_x":"recommenderId","loginId":"count"},inplace=True)

In [112]:
interactions_alg_total_group

,action,recommenderId,count
0,GoToMovieDetails,FilmReleaseDate,108
1,GoToMovieDetails,PredictedRating,109
2,GoToMovieDetails,ShuffledTopPicks,63
3,GoToMovieDetails,TagSimilarity,101
4,MovieDetailsModalOpened,FilmReleaseDate,636
5,MovieDetailsModalOpened,PredictedRating,691
6,MovieDetailsModalOpened,ShuffledTopPicks,404
7,MovieDetailsModalOpened,TagSimilarity,593
8,TrailerDisliked,FilmReleaseDate,211
9,TrailerDisliked,PredictedRating,226


In [121]:
# Action_alg_rec_small is created abooove

actions_alg_total_agg = actions_alg_total_small.groupby('recommenderId').count().reset_index()
actions_alg_total_agg = actions_alg_total_agg[['recommenderId','loginId']].copy()
actions_alg_total_agg.rename(columns={"loginId":"count_total_clicks"},inplace=True)

In [122]:
actions_alg_total_agg.head()

,recommenderId,count_total_clicks
0,FilmReleaseDate,41401
1,PredictedRating,41675
2,ShuffledTopPicks,41154
3,TagSimilarity,42729


In [126]:
interactions_alg_total_percent = pd.merge(interactions_alg_total_group, actions_alg_total_agg, on="recommenderId", how="left")
interactions_alg_total_percent['ratio_action_total_clicks'] = interactions_alg_total_percent['count']/interactions_alg_total_percent['count_total_clicks']

In [127]:
interactions_alg_total_percent

,action,recommenderId,count,count_total_clicks,ratio_action_total_clicks
0,GoToMovieDetails,FilmReleaseDate,108,41401,0.002609
1,GoToMovieDetails,PredictedRating,109,41675,0.002615
2,GoToMovieDetails,ShuffledTopPicks,63,41154,0.001531
3,GoToMovieDetails,TagSimilarity,101,42729,0.002364
4,MovieDetailsModalOpened,FilmReleaseDate,636,41401,0.015362
5,MovieDetailsModalOpened,PredictedRating,691,41675,0.016581
6,MovieDetailsModalOpened,ShuffledTopPicks,404,41154,0.009817
7,MovieDetailsModalOpened,TagSimilarity,593,42729,0.013878
8,TrailerDisliked,FilmReleaseDate,211,41401,0.005096
9,TrailerDisliked,PredictedRating,226,41675,0.005423


In [128]:
interactions_alg_total_percent.to_csv('../Clean_Data/interactions_total_ratio.csv')